## UPAC96 Startup

This notebook demostrates how to startup the UPAC96 and capture an event

### Naludaq Version
*Max Version*: `0.17.2`  
*Min Version*: `0.17.2`

In [ ]:
# Print Naludaq version
import naludaq
print(f"Naludaq version: {naludaq.__version__}")

### Compatible Boards
+ `UPAC96`
    + Firmware: `v911` - `v911`


In [ ]:
%load_ext autoreload
%autoreload 2

from collections import deque
from logging import getLogger, DEBUG, INFO, StreamHandler, Formatter

from naludaq.board.connections._UART import UART
from naludaq.board.connections._FTDI import FTDI
from naludaq.board import Board, startup_board
from naludaq.communication import AnalogRegisters, DigitalRegisters, ControlRegisters
from naludaq.daq.workers.worker_serial_reader import SerialReader

import serial
import time

In [ ]:
ports = [comport for comport in serial.tools.list_ports.comports()]
for port in ports:
    print(port.device)

### Logger

In [ ]:
def setup_logger(connection_level = DEBUG):
    """Setup a logger for Naludaq to print out debug information.
    
    Args:
        connection_level (int): Sets logging level for UART/FTDI connections.
            Warning: If set too low, VS Code may crash
    """
    logger = getLogger()
    logger.setLevel(DEBUG)

    # UART is very verbose, keep on info unless something is broken.
    getLogger("naludaq.UART").setLevel(connection_level)
    getLogger("naludaq.FTDI").setLevel(connection_level)

    handler = StreamHandler()
    handler.setFormatter(Formatter("%(asctime)s %(name)-30s [%(levelname)-6s]: %(message)s"))
    logger.addHandler(handler)

    return logger

try:
    logger
except:
    logger = setup_logger()


## Board Connection

In [ ]:
board = Board('upac96')
board.load_registers()
board.get_ftdi_connection(serial_number='A904CZWB', baud=115200)


### Startup Routine

In [ ]:
startup_board(board)

In [ ]:
cr = ControlRegisters(board)
# Flush fifos
cr.write('usb_fifo_disable', 1)
cr.write('uart_fifo_disable', 1)
time.sleep(0.1)
cr.write('usb_fifo_disable', 0)
cr.write('uart_fifo_disable', 1)
print(f"udc_tx_idle_locked: {cr.read('udc_tx_idle_locked')}")
print(f"udc_rxout_locked: {cr.read('udc_rxout_locked')}")

### Get Ready for Data

In [ ]:
dr = DigitalRegisters(board)
dr.write('enable_testpattern', 1)
cr.write('continuousmode', 1)
cr.write('arm', 1)
time.sleep(0.25)
cr.write('arm', 0)
cr.write('udc_rxout_enable', 0)
cr.write('udc_txin_enable', 0)
time.sleep(0.25)
cr.write('udc_rxout_enable', 0x3F)
cr.write('udc_txin_enable', 0x3F)

### Trigger

In [ ]:
# "00"   Software Trigger
# "01"   UDC Self Trigger
# "10"   FPGA auto trigger 1 Hz
# "11"   External Trigger
cr.write('trigger_select', 0x00)

### Get Data

In [ ]:
buffer = deque()

In [ ]:
sr = SerialReader(board.connection, buffer)
sr.start()
time.sleep(0.1)
board.connection.send('C0000003')
time.sleep(12)
sr.stop()

In [ ]:
data_size = sum([len(packet) for packet in buffer])
data = [packet for packet in buffer]

print(data_size)

In [ ]:
data = b"".join(reversed(data))

In [ ]:
for i in range(0, len(data), 2):
    print(f"{i//2:<6}: {data[i:i+2].hex()}")

In [ ]:
board.disconnect()